### Laboratorio 3 
#### Stefano Aragoni, Carol Arevalo

--------


1. Análisis Exploratorio:
Describir la serie de tiempo y visualizarla

In [26]:
import pandas as pd
import matplotlib.pyplot as plt
from darts import TimeSeries

# Cargar los datos
dataframe = pd.read_csv("shampoo.csv", delimiter=",")

# Se cambia el formato
dataframe["Month"] = pd.to_datetime(dataframe["Month"], format="%m-%y")
dataframe["Mes"] = dataframe["Month"].dt.strftime("%Y-%m")
dataframe = dataframe.drop("Month", axis=1)

# Se cambia el nombre de la columna
dataframe["Ventas"] = dataframe["Sales"].astype(float)
dataframe = dataframe.drop("Sales", axis=1)

# Se imprime el dataframe
dataframe.head(10)

,Mes,Ventas
0,2001-01,266.0
1,2002-01,145.9
2,2003-01,183.1
3,2004-01,119.3
4,2005-01,180.3
5,2006-01,168.5
6,2007-01,231.8
7,2008-01,224.5
8,2009-01,192.8
9,2010-01,122.9


In [25]:
# Cargar la serie de tiempo
ts = TimeSeries.from_dataframe(dataframe, 'Year-Month', 'Sales')

# Visualizar la serie de tiempo
ts.plot(title='Serie de Tiempo')

# Realizar descomposición y visualizar componentes
ts.decompose().plot()

AttributeError: time_col='Year-Month' is not present.


AttributeError: time_col='Year-Month' is not present.

Promedios:
• Aplicar métodos de promedios y comparar los resultados con el conjunto original

In [ ]:
from darts.models import NaiveMean, MovingAverage, ExponentialSmoothing

# Aplicar promedios móviles
naive_mean = NaiveMean()
moving_avg = MovingAverage(window=10)
exp_smoothing = ExponentialSmoothing()

naive_mean.fit(ts)
moving_avg.fit(ts)
exp_smoothing.fit(ts)

# Visualizar resultados
ts.plot(label='Original')
naive_mean.predict(len(ts)).plot(label='Naive Mean')
moving_avg.predict(len(ts)).plot(label='Moving Average')
exp_smoothing.predict(len(ts)).plot(label='Exponential Smoothing')
plt.legend()
plt.show()


SARIMA:
• Identificar parámetros y ajustar un modelo SARIMA

In [ ]:
from darts.models import SARIMA

sarima_model = SARIMA(p=1, d=1, q=1, P=1, D=1, Q=1, s=12)
sarima_model.fit(ts)
sarima_forecast = sarima_model.predict(len(ts))

ts.plot(label='Original')
sarima_forecast.plot(label='SARIMA Forecast')
plt.legend()
plt.show()


Alisamiento Exponencial:
• Aplicar diferentes métodos de alisamiento exponencial y comparar.

In [ ]:
from darts.models import SimpleExponentialSmoothing, HoltWinters

ses = SimpleExponentialSmoothing()
holt = Holt()
holt_winters = HoltWinters()

ses.fit(ts)
holt.fit(ts)
holt_winters.fit(ts)

ts.plot(label='Original')
ses.predict(len(ts)).plot(label='Simple Exponential Smoothing')
holt.predict(len(ts)).plot(label='Holt')
holt_winters.predict(len(ts)).plot(label='Holt-Winters')
plt.legend()
plt.show()


Prophet:
• Utilizar Prophet para modelar la serie de tiempo.

In [ ]:
from fbprophet import Prophet

prophet_model = Prophet()
df_prophet = ts.pd_dataframe()
df_prophet.rename(columns={'fecha': 'ds', 'valor': 'y'}, inplace=True)

prophet_model.fit(df_prophet)
future = prophet_model.make_future_dataframe(periods=forecast_horizon)
forecast = prophet_model.predict(future)

prophet_model.plot(forecast)
plt.show()


Redes Neuronales:
• Implementar una red neuronal simple para prever la serie de tiempo

In [ ]:
from darts.models import RNNModel
from darts.utils.timeseries_generation import gaussian_timeseries

# Crear una serie de tiempo sintética (en caso de que no se tenga una serie real)
synthetic_ts = gaussian_timeseries(length=len(ts), freq='D', start_ts=ts.time_index[0])

rnn_model = RNNModel(input_chunk_length=32, output_chunk_length=10, n_epochs=100)
rnn_model.fit(synthetic_ts)
rnn_forecast = rnn_model.predict(forecast_horizon)

synthetic_ts.plot(label='Sintético')
rnn_forecast.plot(label='RNN Forecast')
plt.legend()
plt.show()


Comparación y Evaluación:
• Usar métricas como RMSE, MAE para comparar los modelos.
• Discutir cuál algoritmo se desempeña mejor para cada tipo de conjunto de datos y
por qué

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Calcular métricas de evaluación para cada modelo
rmse_sarima = mean_squared_error(ts, sarima_forecast, squared=False)
mae_sarima = mean_absolute_error(ts, sarima_forecast)

rmse_rnn = mean_squared_error(synthetic_ts, rnn_forecast, squared=False)
mae_rnn = mean_absolute_error(synthetic_ts, rnn_forecast)

# Imprimir resultados
print(f'SARIMA RMSE: {rmse_sarima}, SARIMA MAE: {mae_sarima}')
print(f'RNN RMSE: {rmse_rnn}, RNN MAE: {mae_rnn}')
